### Imports

In [6]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip


### Initializing YOLO Network using predefined weight and config

In [ ]:
weights_path = './yolo/yolov4.weights'
config_path = './yolo/yolov4.cfg'
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

### Loading names of classes detected by YOLO

In [ ]:
labels_path = './yolo/coco.names'
names = net.getLayerNames()
layers_names = [names[i - 1] for i in net.getUnconnectedOutLayers()]
labels = open(labels_path).read().strip().split("\n")


### Model inferring and drawing bounding boxes function

In [ ]:
def detect_yolo(img):
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=False, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(layers_names)
    boxes, confidences, class_ids = [], [], []
    (H, W) = img.shape[:2]
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)
    idxArr = np.asarray(idxs)
    for n in idxArr.flatten():
        (x, y) = (boxes[n][0], boxes[n][1])
        (w, h) = (boxes[n][2], boxes[n][3])        
        cv2.rectangle(img, (x, y), (x + w, y + h), (255,255,0), 2)
        text = "{}: {:.4f}".format(labels[class_ids[n]], confidences[n])
        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)
    return img



### Loading test image

In [ ]:
image_path = './yolo/test_image.jpg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img = detect_yolo(img)
plt.imshow(img)

### Video Output

In [ ]:
project_video = VideoFileClip('./project_video.mp4')
white_clip = project_video.fl_image(detect_yolo)
white_clip.write_videofile('./out_note.mp4', audio=False,threads=4, preset='ultrafast')